In [1]:
import tensorflow as tf
from tqdm import tqdm
import numpy as np
import os 
import zipfile
from tensorflow.keras.optimizers.schedules import PiecewiseConstantDecay
from utils_speedtraining import *



In [2]:
from model_ssd300 import SSD300
from dataset_calling import ReceiptDetection
from train_ssd import train
from utils_speedtraining import *
from vgg16_redefined import VGG16_backbone

In [3]:
! pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d dhiaznaidi/receiptdatasetssd300v2

 98% 425M/432M [00:05<00:00, 90.1MB/s]
100% 432M/432M [00:06<00:00, 73.0MB/s]


In [3]:
cwd = os.getcwd()
zip_ref = zipfile.ZipFile(cwd+"/receiptdatasetssd300v2.zip", 'r')
zip_ref.extractall(cwd)
zip_ref.close()

In [4]:
cwd = os.getcwd()

In [5]:
dataset_path = cwd+"/fullDataset"

In [6]:
preprocessed_data_path = cwd+"/train_preprocessed_data"
saved_weights_path = cwd+"/saved_weights"

## Load SSD300 Model and VGG16 weights

In [7]:
SSD300_model = SSD300(3, floatType=32)
SSD300_model.build(input_shape=(32, 300, 300, 3))
SSD300_model.load_vgg16_imagenet_weights()

In [ ]:
SSD300_model.getModel().summary()

In [9]:
calling_data = ReceiptDetection(dataset_path, batch_size=16, floatType=32)

In [14]:
init_lr = 0.001
lr_decay = PiecewiseConstantDecay(
    boundaries=[80, 170, 260],
    values=[init_lr, 0.0005, 0.0001, 0.00005])

optimizer = tf.keras.optimizers.SGD(
    learning_rate=lr_decay,
    momentum=0.9)

In [ ]:
# Preprocess data
# to be done only once,
# if the path exists: assuming that the preprocessing has already been done completely
if not os.path.exists(preprocessed_data_path):
    saveGTdata(dataset_path, preprocessed_data_path)

In [ ]:
imgs, confs, locs = loadGTdata(preprocessed_data_path, nb_data_to_load=500)

In [58]:
os.mkdir(saved_weights_path)

In [ ]:
train(SSD300_model, optimizer,
      imgs, confs, locs, weights_path=saved_weights_path,
      num_epoch=35, inter_save=5)